In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from dmba import classificationSummary

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt
import numpy as np 
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from sklearn.ensemble  import VotingClassifier
from sklearn.metrics import accuracy_score

# test

In [2]:
test_merge_data = pd.read_csv('test_merge_data.csv')
pd.set_option('display.max_columns',None)
test_merge_data


#family 변수 묶어주기 (은호님 코드)
test_merge_data['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)

test_merge_data['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)

test_merge_data['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)

test_merge_data['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)

test_merge_data['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

#이진형
test_merge_data['NewYearsDay'] = np.where(test_merge_data['NewYearsDay'] == 'True',1,0)

#worday변수 생성
test_merge_data['workday'] = 1 #true: 1 ,false: 0 

test_merge_data.loc[test_merge_data.Weekday > 4, 'workday'] = 0

#get dummies
test_merge_data = pd.get_dummies(test_merge_data,columns=['family','type_y']) 

In [3]:
#버릴변수
test_merge_data.drop(columns=['Unnamed: 0','id','date','type_x','locale','locale_name','description','transferred','city','state'],inplace=True)
pd.set_option('display.max_columns',None)
test_merge_data

,store_nbr,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_BEVERAGES,family_BREAD/BAKERY,family_CLEANING,family_DAIRY,family_Daily,family_FROZEN FOODS,family_Food,family_GROCERY I,family_Home,family_LifeStyle,family_MEATS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_Tools,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E
0,1,0,46.80,1448.259579,13,2017,8,0,2,1688,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,1,0,46.80,1448.259579,13,2017,8,0,2,1688,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,1,2,46.80,1448.259579,13,2017,8,0,2,1688,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,20,46.80,1448.259579,13,2017,8,0,2,1688,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,0,46.80,1448.259579,13,2017,8,0,2,1688,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,9,1,47.26,1986.976875,6,2017,8,0,3,1703,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
28508,9,0,47.26,1986.976875,6,2017,8,0,3,1703,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
28509,9,1,47.26,1986.976875,6,2017,8,0,3,1703,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
28510,9,9,47.26,1986.976875,6,2017,8,0,3,1703,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


# train

In [4]:
train_merge_data = pd.read_csv('train_merge_data.csv')
pd.set_option('display.max_columns',None)
train_merge_data.head()
train_merge_data.isnull().sum()

C:\Users\rhskr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Unnamed: 0      0
id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
dcoilwtico      0
type_x          0
locale          0
locale_name     0
description     0
transferred     0
transactions    0
city            0
state           0
type_y          0
cluster         0
Year            0
Month           0
Weekend         0
Weekday         0
trend           0
NewYearsDay     0
dtype: int64

In [5]:
train_merge_data['workday'] = 1 #true: 1 ,false: 0 

train_merge_data.loc[train_merge_data.Weekday > 4, 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Bridge'  , 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Work Day', 'workday'] = 1
train_merge_data.loc[train_merge_data.type_x == 'Transfer', 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == False), 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == True ), 'workday'] = 1

#family 변수 묶어주기 (은호님 코드)
train_merge_data['family'].replace(['PRODUCE','AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)
#produce가 tools로 가도 되나 ..?
train_merge_data['family'].replace(['BEAUTY', 'LINGERIE','CLEANING', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)

train_merge_data['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)

train_merge_data['family'].replace(['GROCERY II','FROZEN FOODS','MEATS','POULTRY','PREPARED FOODS','BREAD/BAKERY','GROCERY I', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)

train_merge_data['family'].replace(['LIQUOR', 'WINE,BEER','DAIRY','BEVERAGES'], 'Drink', inplace=True)
#dairy food에 넣어야 하나?
train_merge_data['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)


In [6]:
train_merge_data['family'].value_counts()

Food         925560
LifeStyle    833004
Tools        462780
Home         462780
Drink        185112
Daily        185112
Name: family, dtype: int64

In [7]:
#get dummies
train_merge_data.loc[train_merge_data['description'].str.contains('Terremoto', na=False),
                 'description'] = 'Earthquake'
train_merge_data.loc[train_merge_data['description'].str.contains('futbol', na=False), 
                 'description'] = 'Football'

train_merge_data = pd.get_dummies(train_merge_data,columns=['family','type_x','locale','type_y']) 

descriptions = pd.get_dummies(train_merge_data['description'])[['Earthquake', 'Cyber Monday', 'Black Friday']]
train_merge_data = train_merge_data.merge(descriptions, how='left', left_index=True, right_index=True)

#이진형
train_merge_data['NewYearsDay'] = np.where(train_merge_data['NewYearsDay'] == 'True',1,0)

In [8]:
pd.set_option('display.max_columns',None)
train_merge_data.head()
#type_x_Empty .... 

,Unnamed: 0,id,date,store_nbr,sales,onpromotion,dcoilwtico,locale_name,description,transferred,transactions,city,state,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_Daily,family_Drink,family_Food,family_Home,family_LifeStyle,family_Tools,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday
0,0,0,2013-01-01,1,0.0,0,93.14,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,13,2013,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,1,2013-01-01,1,0.0,0,93.14,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,2,2,2013-01-01,1,0.0,0,93.14,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,3,3,2013-01-01,1,0.0,0,93.14,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,4,4,2013-01-01,1,0.0,0,93.14,Ecuador,Primer dia del ano,False,1420.876311,Quito,Pichincha,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [9]:
#버릴 변수
train_merge_data.drop(columns=['Unnamed: 0','id','description','locale_name','transferred','city','state'],inplace=True)

In [10]:
train_merge_data.head()

,date,store_nbr,sales,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_Daily,family_Drink,family_Food,family_Home,family_LifeStyle,family_Tools,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday
0,2013-01-01,1,0.0,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,2013-01-01,1,0.0,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,2013-01-01,1,0.0,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,2013-01-01,1,0.0,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,2013-01-01,1,0.0,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [11]:
#train,valid나누기
all_train = train_merge_data[train_merge_data.date < '2016-08-01']

all_valid = train_merge_data[train_merge_data.date >= '2016-08-01']

X_train = all_train.drop(columns=['sales','date'])
y_train = all_train['sales']


X_valid = all_valid.drop(columns=['sales','date'])
y_valid = all_valid['sales']

In [12]:
y_train = y_train.astype('int')
y_valid = y_valid.astype('int')

In [14]:
#분류나무모델
fullClassTree = DecisionTreeClassifier()
fullClassTree.fit(X_train, y_train)

pred_train = fullClassTree.predict(X_train)
pred_train = np.where(pred_train<0,0,pred_train)
pred_valid = fullClassTree.predict(X_valid)
pred_valid = np.where(pred_valid<0,0,pred_valid)

acc_train = accuracy_score(y_train, pred_train)
acc_valid = accuracy_score(y_valid, pred_valid)

MemoryError: could not allocate 25920798720 bytes

In [ ]:
#로지스틱 회귀모형 
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(X_train, y_train)

pred_train = logit_reg.predict(X_train)
pred_valid = logit_reg.predict(X_valid)

acc_train = accuracy_score(y_train, pred_train)
acc_valid = accuracy_score(y_valid, pred_valid)

In [ ]:
#SVM
#https://blog.naver.com/winddori2002/221667083964
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate


#scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
scaler.fit(X_valid)
X_valid = scaler.transform(X_valid)

svm_clf =svm.SVC(kernel = 'linear', random_state=100) #선형분리

scores = cross_val_score(svm_clf, X_train, y_train, cv = 5)
scores

from sklearn.model_selection import GridSearchCV

# 테스트하고자 하는 파라미터 값들을 사전타입으로 정의

svm_clf = svm.SVC(kernel = 'linear',random_state=100)
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}

grid_svm = GridSearchCV(svm_clf,
                      param_grid = parameters, cv = 5)

grid_svm.fit(X_train, y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)


In [ ]:
# ridge
ridge_alpha = 1

ridge = Ridge(alpha = ridge_alpha)
ridge.fit(X_train,y_train)

pred_train = ridge.predict(X_train)
pred_train = np.where(pred_train<0,0,pred_train)
pred_valid = ridge.predict(X_valid)
pred_valid = np.where(pred_valid<0,0,pred_valid)

acc_train = accuracy_score(y_train, pred_train)
acc_valid = accuracy_score(y_valid, pred_valid)

In [ ]:
#lasso
lasso_alpha = 0.1

lasso = Lasso(alpha = lasso_alpha)
lasso.fit(X_train,y_train)

pred_train = lasso.predict(X_train)
pred_train = np.where(pred_train<0,0,pred_train)
pred_valid = lasso.predict(X_valid)
pred_valid = np.where(pred_valid<0,0,pred_valid)

acc_train = accuracy_score(y_train, pred_train)
acc_valid = accuracy_score(y_valid, pred_valid)

https://data-newbie.tistory.com/446

chi-square 독립성 검정으로 범주형 변수 선택하기